# Colaborating Filterting Methods on User and Item
## Jaccard Score and Pearson Correlations 

In [43]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.metrics import jaccard_score

In [44]:
users = ['User1', 'User2', 'User3', 'User4', 'User5']
items = ['Item A', 'Item B', 'Item C', 'Item D', 'Item E']

datasets = [
    [1, 0, 1, 1, 0],
    [1, 0, 0, 1, 1],
    [1, 0, 1, 0, 0],
    [0, 1, 0, 1, 1],
    [1, 1, 1, 0, 0],
]

df = pd.DataFrame(datasets, columns=items, index=users)

# Calculate Jaccard similarity for items
item_jaccard_similarity_matrix = pd.DataFrame(0, index=items, columns=items, dtype=float)

for col1 in items:
    for col2 in items:
        item_jaccard_similarity_matrix.loc[col1, col2] = jaccard_score(df[col1], df[col2])

print("Item Jaccard Similarity Matrix:")
print(item_jaccard_similarity_matrix)

# Calculate Jaccard similarity for users
user_jaccard_similarity_matrix = pd.DataFrame(0, index=users, columns=users, dtype=float)

for user1 in users:
    for user2 in users:
        user_jaccard_similarity_matrix.loc[user1, user2] = jaccard_score(df.loc[user1], df.loc[user2])

print("\nUser Jaccard Similarity Matrix:")
print(user_jaccard_similarity_matrix)

Item Jaccard Similarity Matrix:
        Item A    Item B  Item C    Item D    Item E
Item A    1.00  0.200000    0.75  0.400000  0.200000
Item B    0.20  1.000000    0.25  0.250000  0.333333
Item C    0.75  0.250000    1.00  0.200000  0.000000
Item D    0.40  0.250000    0.20  1.000000  0.666667
Item E    0.20  0.333333    0.00  0.666667  1.000000

User Jaccard Similarity Matrix:
          User1  User2     User3  User4     User5
User1  1.000000   0.50  0.666667    0.2  0.500000
User2  0.500000   1.00  0.250000    0.5  0.200000
User3  0.666667   0.25  1.000000    0.0  0.666667
User4  0.200000   0.50  0.000000    1.0  0.200000
User5  0.500000   0.20  0.666667    0.2  1.000000


In [45]:
# find similar users based similarities, except itself
topN_users = {}
for i in user_jaccard_similarity_matrix.index:
    _df = user_jaccard_similarity_matrix.loc[i].drop([i])
    _df_sorted = _df.sort_values(ascending=False)
    top2 = list(_df_sorted.index[:2])
    topN_users[i] = top2

In [46]:
topN_users

{'User1': ['User3', 'User2'],
 'User2': ['User1', 'User4'],
 'User3': ['User1', 'User5'],
 'User4': ['User2', 'User1'],
 'User5': ['User3', 'User1']}

In [47]:
rs_results = {}
for user, sim_users in topN_users.items():
    rs_result = set()
    for sim_user in sim_users:
        rs_result = rs_result.union(set(df.loc[sim_user].replace(0,np.nan).dropna().index))
        
    rs_result -= set(df.loc[user].replace(0,np.nan).dropna().index)
    rs_results[user] = rs_result 
    
rs_results

{'User1': {'Item E'},
 'User2': {'Item B', 'Item C'},
 'User3': {'Item B', 'Item D'},
 'User4': {'Item A', 'Item C'},
 'User5': {'Item D'}}

In [50]:
# ratings case 
users = ['User1', 'User2', 'User3', 'User4', 'User5']
items = ['Item A', 'Item B', 'Item C', 'Item D', 'Item E']

datasets = [
    [5, 3, 4, 4, None],
    [3, 1, 2, 3, 3],
    [4, 3, 4, 3, 5],
    [3, 3, 1, 5, 4],
    [1, 5, 5, 2, 1],
]

df = pd.DataFrame(datasets, columns=items, index=users)
user_similar = df.T.corr()
item_similar = df.corr()
print("user corrlation")
print(user_similar.round(4))
print("item corrlation")
print(item_similar.round(4))

user corrlation
        User1   User2   User3   User4   User5
User1  1.0000  0.8528  0.7071  0.0000 -0.7921
User2  0.8528  1.0000  0.4677  0.4900 -0.9001
User3  0.7071  0.4677  1.0000 -0.1612 -0.4666
User4  0.0000  0.4900 -0.1612  1.0000 -0.6415
User5 -0.7921 -0.9001 -0.4666 -0.6415  1.0000
item corrlation
        Item A  Item B  Item C  Item D  Item E
Item A  1.0000 -0.4767 -0.1231  0.5322  0.9695
Item B -0.4767  1.0000  0.6455 -0.3101 -0.4781
Item C -0.1231  0.6455  1.0000 -0.7206 -0.4276
Item D  0.5322 -0.3101 -0.7206  1.0000  0.5817
Item E  0.9695 -0.4781 -0.4276  0.5817  1.0000


# Matrix Decomposition

In [54]:
import os
import pandas as pd
import numpy as np

In [55]:
# Get the current working directory
current_directory = os.getcwd()

print("Current Working Directory:", current_directory)

Current Working Directory: /Users/dunpeigan


In [58]:
data_path = "Downloads/movies/ratings.csv"
cache_dir = 'Downloads/movies/cache/'

In [60]:
def load_data(data_path):
    cache_path = os.path.join(cache_dir, "ratings_matrix.cache")
    
    print('start to load datasets')
    if os.path.exists(cache_path):
        print('start cache')
        ratings_matrix = pd.read_pickle(cache_path)
        print('cache dataset complete')
    else:
        print('load new dataset')
        dtype = {'userId': np.int32, 'movieId':np.int32, 'rating':np.float32}
        ratings = pd.read_csv(data_path, dtype=dtype, usecols=range(3))
        ratings_matrix = ratings.pivot_table(index=['userId'],columns=['movieId'],values='rating')
        ratings_matrix.to_pickle(cache_path)
        print('dataset loaded')
    return ratings_matrix

if __name__ == '__main__':
    ratings_matrix = load_data(data_path)
    print(ratings_matrix)

start to load datasets
load new dataset
dataset loaded
movieId  1       2       3       4       5       6       7       8       \
userId                                                                    
1           4.0     NaN     4.0     NaN     NaN     4.0     NaN     NaN   
2           NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
3           NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
4           NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
5           4.0     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
...         ...     ...     ...     ...     ...     ...     ...     ...   
606         2.5     NaN     NaN     NaN     NaN     NaN     2.5     NaN   
607         4.0     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
608         2.5     2.0     2.0     NaN     NaN     NaN     NaN     NaN   
609         3.0     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
610         5.0     NaN     NaN     NaN     N

In [65]:
def compute_pearson_similarity(ratings_matrix, based='user'):
    user_similarity_cache_path = os.path.join(cache_dir, 'user_similarity.cache')
    item_similarity_cache_path = os.path.join(cache_dir, 'item_similarity.cache')
    if based == 'user':
        if os.path.exists(user_similarity_cache_path):
            print('load user similarity matrix from cache')
            similarity = pd.read_pickle(user_similarity_cache_path)
        else:
            print('start to compute user similarity matrix')
            similarity = ratings_matrix.T.corr()
            similarity.to_pickle(user_similarity_cache_path)
    elif based=='item':
        if os.path.exists(item_similarity_cache_path):
            print('load item similarity matrix from cache')
            similarity = pd.read_pickle(item_similarity_cache_path)
        else:
            print('start to compute item similarity matrix')
            similarity = ratings_matrix.corr()
            similarity.to_pickle(item_similarity_cache_path)
    else:
        raise Exception("Unhandled 'based' Values: %s"%based)
    print("similarity matrix computed/loaded")
    return similarity

if __name__ == '__main__':
    ratings_matrix = load_data(data_path)
    user_similar = compute_pearson_similarity(ratings_matrix, 'user')
    print(user_similar)
    item_similar = compute_pearson_similarity(ratings_matrix, 'item')
    print(item_similar)
        
                

start to load datasets
start cache
cache dataset complete
start to compute user similarity matrix
similarity matrix computed/loaded
userId       1         2         3         4         5         6         7    \
userId                                                                         
1       1.000000       NaN  0.079819  0.207983  0.268749 -0.291636 -0.118773   
2            NaN  1.000000       NaN       NaN       NaN       NaN -0.991241   
3       0.079819       NaN  1.000000       NaN       NaN       NaN       NaN   
4       0.207983       NaN       NaN  1.000000 -0.336525  0.148498  0.542861   
5       0.268749       NaN       NaN -0.336525  1.000000  0.043166  0.158114   
...          ...       ...       ...       ...       ...       ...       ...   
606     0.066378  0.583333 -0.791334  0.144603  0.244321 -0.049192  0.137771   
607     0.174557       NaN -0.333333  0.116518  0.231080  0.255639  0.402792   
608     0.268070 -0.125000 -0.395092 -0.170501 -0.020546  0.125428  

In [67]:
def predict(uid, iid, ratings_matrix, user_similar):
    print("start to predict user<%d>'s rating for movie<%d>"%(uid, iid))
    # find similar users as the uid user
    similar_users = user_similar[uid].drop([uid]).dropna()
    # positive correlated users 
    similar_users = similar_users.where(similar_users>0).dropna()
    if similar_users.empty is True:
        raise Exception("user<%d> has no similar user"% uid)
        
    # find the users who has ratings for iid movie
    ids = set(ratings_matrix[iid].dropna().index)&set(similar_users.index)
    finally_similar_users = similar_users.loc[list(ids)]
    
    # combine the correlations for close users to have prediction for iid movie
    sum_up = 0
    sum_down = 0
    for sim_uid, similarity in finally_similar_users.iteritems():
        # close users ratings 
        sim_user_rated_movies = ratings_matrix.loc[sim_uid].dropna()
        sim_user_rating_for_item = sim_user_rated_movies[iid]
        sum_up += similarity * sim_user_rating_for_item
        sum_down += similarity
    predict_rating = sum_up/sum_down
    print("predict user<%d> rating for movie<%d>: %0.2f" % (uid, iid, predict_rating))
    return round(predict_rating, 2)

if __name__ == '__main__':
    rating_matrix = load_data(data_path)
    
    user_similar = compute_pearson_similarity(ratings_matrix, based="user")
    predict(1,1,ratings_matrix, user_similar)
    predict(1,2,ratings_matrix, user_similar)

start to load datasets
start cache
cache dataset complete
load user similarity matrix from cache
similarity matrix computed/loaded
start to predict user<1>'s rating for movie<1>
predict user<1> rating for movie<1>: 3.87
start to predict user<1>'s rating for movie<2>
predict user<1> rating for movie<2>: 3.38


In [70]:
def predict_all(uid, ratings_matrix, user_similar):
    item_ids = ratings_matrix.columns
    for iid in item_ids:
        try:
            rating = predict(uid, iid, ratings_matrix, user_similar)
        except Exception as e:
            print(e)
        else:
            yield uid, iid, rating


if __name__ == '__main__':
    ratings_matrix = load_data(data_path)
    user_similar = compute_pearson_similarity(ratings_matrix, based="user")
    for i in predict_all(1, ratings_matrix, user_similar):
        pass 
    

start to load datasets
start cache
cache dataset complete
load user similarity matrix from cache
similarity matrix computed/loaded
start to predict user<1>'s rating for movie<1>
predict user<1> rating for movie<1>: 3.87
start to predict user<1>'s rating for movie<2>
predict user<1> rating for movie<2>: 3.38
start to predict user<1>'s rating for movie<3>
predict user<1> rating for movie<3>: 3.14
start to predict user<1>'s rating for movie<4>
predict user<1> rating for movie<4>: 2.41
start to predict user<1>'s rating for movie<5>
predict user<1> rating for movie<5>: 3.09
start to predict user<1>'s rating for movie<6>
predict user<1> rating for movie<6>: 3.83
start to predict user<1>'s rating for movie<7>
predict user<1> rating for movie<7>: 3.22
start to predict user<1>'s rating for movie<8>
predict user<1> rating for movie<8>: 2.47
start to predict user<1>'s rating for movie<9>
predict user<1> rating for movie<9>: 2.59
start to predict user<1>'s rating for movie<10>
predict user<1> rati

predict user<1> rating for movie<106>: 4.00
start to predict user<1>'s rating for movie<107>
predict user<1> rating for movie<107>: 3.10
start to predict user<1>'s rating for movie<108>
division by zero
start to predict user<1>'s rating for movie<110>
predict user<1> rating for movie<110>: 4.04
start to predict user<1>'s rating for movie<111>
predict user<1> rating for movie<111>: 3.99
start to predict user<1>'s rating for movie<112>
predict user<1> rating for movie<112>: 3.48
start to predict user<1>'s rating for movie<113>
division by zero
start to predict user<1>'s rating for movie<116>
predict user<1> rating for movie<116>: 4.34
start to predict user<1>'s rating for movie<117>
division by zero
start to predict user<1>'s rating for movie<118>
predict user<1> rating for movie<118>: 2.44
start to predict user<1>'s rating for movie<119>
predict user<1> rating for movie<119>: 3.00
start to predict user<1>'s rating for movie<121>
division by zero
start to predict user<1>'s rating for mov

predict user<1> rating for movie<247>: 3.89
start to predict user<1>'s rating for movie<248>
predict user<1> rating for movie<248>: 2.79
start to predict user<1>'s rating for movie<249>
predict user<1> rating for movie<249>: 3.81
start to predict user<1>'s rating for movie<250>
predict user<1> rating for movie<250>: 2.14
start to predict user<1>'s rating for movie<251>
predict user<1> rating for movie<251>: 3.31
start to predict user<1>'s rating for movie<252>
predict user<1> rating for movie<252>: 3.49
start to predict user<1>'s rating for movie<253>
predict user<1> rating for movie<253>: 3.36
start to predict user<1>'s rating for movie<254>
division by zero
start to predict user<1>'s rating for movie<255>
predict user<1> rating for movie<255>: 1.26
start to predict user<1>'s rating for movie<256>
predict user<1> rating for movie<256>: 2.66
start to predict user<1>'s rating for movie<257>
predict user<1> rating for movie<257>: 3.36
start to predict user<1>'s rating for movie<258>
pred

predict user<1> rating for movie<347>: 4.47
start to predict user<1>'s rating for movie<348>
predict user<1> rating for movie<348>: 3.64
start to predict user<1>'s rating for movie<349>
predict user<1> rating for movie<349>: 3.59
start to predict user<1>'s rating for movie<350>
predict user<1> rating for movie<350>: 3.07
start to predict user<1>'s rating for movie<351>
predict user<1> rating for movie<351>: 3.22
start to predict user<1>'s rating for movie<352>
predict user<1> rating for movie<352>: 2.00
start to predict user<1>'s rating for movie<353>
predict user<1> rating for movie<353>: 3.90
start to predict user<1>'s rating for movie<354>
predict user<1> rating for movie<354>: 2.91
start to predict user<1>'s rating for movie<355>
predict user<1> rating for movie<355>: 1.99
start to predict user<1>'s rating for movie<356>
predict user<1> rating for movie<356>: 4.18
start to predict user<1>'s rating for movie<357>
predict user<1> rating for movie<357>: 3.55
start to predict user<1>'s

predict user<1> rating for movie<457>: 4.07
start to predict user<1>'s rating for movie<458>
predict user<1> rating for movie<458>: 2.82
start to predict user<1>'s rating for movie<459>
predict user<1> rating for movie<459>: 2.19
start to predict user<1>'s rating for movie<460>
predict user<1> rating for movie<460>: 2.43
start to predict user<1>'s rating for movie<461>
predict user<1> rating for movie<461>: 2.68
start to predict user<1>'s rating for movie<464>
predict user<1> rating for movie<464>: 2.17
start to predict user<1>'s rating for movie<466>
predict user<1> rating for movie<466>: 2.74
start to predict user<1>'s rating for movie<467>
division by zero
start to predict user<1>'s rating for movie<468>
predict user<1> rating for movie<468>: 3.12
start to predict user<1>'s rating for movie<469>
predict user<1> rating for movie<469>: 3.32
start to predict user<1>'s rating for movie<470>
predict user<1> rating for movie<470>: 1.00
start to predict user<1>'s rating for movie<471>
pred

...

division by zero
start to predict user<1>'s rating for movie<140174>
predict user<1> rating for movie<140174>: 3.76
start to predict user<1>'s rating for movie<140237>
predict user<1> rating for movie<140237>: 2.00
start to predict user<1>'s rating for movie<140247>
predict user<1> rating for movie<140247>: 2.82
start to predict user<1>'s rating for movie<140265>
predict user<1> rating for movie<140265>: 5.00
start to predict user<1>'s rating for movie<140267>
division by zero
start to predict user<1>'s rating for movie<140289>
predict user<1> rating for movie<140289>: 4.00
start to predict user<1>'s rating for movie<140301>
predict user<1> rating for movie<140301>: 3.00
start to predict user<1>'s rating for movie<140359>
predict user<1> rating for movie<140359>: 4.34
start to predict user<1>'s rating for movie<140481>
predict user<1> rating for movie<140481>: 4.50
start to predict user<1>'s rating for movie<140523>
predict user<1> rating for movie<140523>: 3.50
start to predict user<1

predict user<1> rating for movie<149354>: 2.65
start to predict user<1>'s rating for movie<149380>
predict user<1> rating for movie<149380>: 1.50
start to predict user<1>'s rating for movie<149406>
predict user<1> rating for movie<149406>: 3.78
start to predict user<1>'s rating for movie<149508>
predict user<1> rating for movie<149508>: 5.00
start to predict user<1>'s rating for movie<149566>
predict user<1> rating for movie<149566>: 5.00
start to predict user<1>'s rating for movie<149590>
predict user<1> rating for movie<149590>: 2.50
start to predict user<1>'s rating for movie<149612>
predict user<1> rating for movie<149612>: 2.00
start to predict user<1>'s rating for movie<149830>
predict user<1> rating for movie<149830>: 4.50
start to predict user<1>'s rating for movie<149902>
predict user<1> rating for movie<149902>: 2.00
start to predict user<1>'s rating for movie<150254>
predict user<1> rating for movie<150254>: 2.00
start to predict user<1>'s rating for movie<150401>
predict us

predict user<1> rating for movie<157110>: 4.50
start to predict user<1>'s rating for movie<157122>
predict user<1> rating for movie<157122>: 3.00
start to predict user<1>'s rating for movie<157130>
predict user<1> rating for movie<157130>: 2.00
start to predict user<1>'s rating for movie<157172>
predict user<1> rating for movie<157172>: 0.50
start to predict user<1>'s rating for movie<157200>
predict user<1> rating for movie<157200>: 2.49
start to predict user<1>'s rating for movie<157270>
predict user<1> rating for movie<157270>: 2.50
start to predict user<1>'s rating for movie<157296>
predict user<1> rating for movie<157296>: 3.31
start to predict user<1>'s rating for movie<157312>
predict user<1> rating for movie<157312>: 1.93
start to predict user<1>'s rating for movie<157340>
predict user<1> rating for movie<157340>: 2.50
start to predict user<1>'s rating for movie<157369>
division by zero
start to predict user<1>'s rating for movie<157407>
predict user<1> rating for movie<157407>

predict user<1> rating for movie<165639>: 3.54
start to predict user<1>'s rating for movie<165645>
predict user<1> rating for movie<165645>: 0.50
start to predict user<1>'s rating for movie<165671>
predict user<1> rating for movie<165671>: 4.50
start to predict user<1>'s rating for movie<165843>
predict user<1> rating for movie<165843>: 4.00
start to predict user<1>'s rating for movie<165947>
predict user<1> rating for movie<165947>: 3.44
start to predict user<1>'s rating for movie<165959>
predict user<1> rating for movie<165959>: 5.00
start to predict user<1>'s rating for movie<165969>
division by zero
start to predict user<1>'s rating for movie<166015>
predict user<1> rating for movie<166015>: 3.50
start to predict user<1>'s rating for movie<166024>
predict user<1> rating for movie<166024>: 4.59
start to predict user<1>'s rating for movie<166183>
predict user<1> rating for movie<166183>: 5.00
start to predict user<1>'s rating for movie<166203>
predict user<1> rating for movie<166203>

predict user<1> rating for movie<174045>: 4.00
start to predict user<1>'s rating for movie<174053>
predict user<1> rating for movie<174053>: 4.86
start to predict user<1>'s rating for movie<174055>
predict user<1> rating for movie<174055>: 3.93
start to predict user<1>'s rating for movie<174141>
division by zero
start to predict user<1>'s rating for movie<174403>
division by zero
start to predict user<1>'s rating for movie<174479>
division by zero
start to predict user<1>'s rating for movie<174551>
predict user<1> rating for movie<174551>: 5.00
start to predict user<1>'s rating for movie<174681>
predict user<1> rating for movie<174681>: 3.50
start to predict user<1>'s rating for movie<174727>
predict user<1> rating for movie<174727>: 3.00
start to predict user<1>'s rating for movie<174737>
division by zero
start to predict user<1>'s rating for movie<174815>
predict user<1> rating for movie<174815>: 3.50
start to predict user<1>'s rating for movie<174909>
predict user<1> rating for movi

In [72]:
def _predict_all(uid, item_ids, ratings_matrix, user_similar):
    for iid in item_ids:
        try:
            rating = predict(uid, iid, ratings_matrix, user_similar)
        except Exception as e:
            print(e)
        else:
            yield uid, iid, rating

def predict_all(uid, ratings_matrix, user_similar, filter_rule = None):
    if not filter_rule:
        items_ids = ratings_matrix.columns
    elif isinstance(filter_rule, str) and filter_rule == "unhot":
        # count the ratings of every movie
        count = ratings_matrix.count()
        # filter ratings larget than 10 as the hot movie
        item_ids = count.where(count>10).dropna().index
    elif isinstance(filter_rule, str) and filter_rule == "rated":
        # get the raings for all movies of a user
        user_ratings = ratings_matrix.loc[uid]
        # less than 6 is rated, else not rated
        _ = user_ratings<6
        item_ids = _.where(_==False).dropna().index
    elif isinstance(filter_rule, list) and set(filter_rule) == set(["unhot","rated"]):
        count = ratings_matrix.count()
        ids1 = count.where(count>10).dropna().index
        user_ratings = ratings_matrix.loc[uid]
        _ = user_ratings < 6
        ids2 = _.where(_ == False).dropna().index
        item_ids = set(ids1)&set(ids2)
    else:
        raise Exception("useless filter parameter")
    
    yield from _predict_all(uid, item_ids, ratings_matrix, user_similar)
    
if __name__ == '__main__':
    ratings_matrix = load_data(data_path)
    user_similar = compute_pearson_similarity(ratings_matrix, based="user")
    for result in predict_all(1, ratings_matrix, user_similar, filter_rule =["unhot", "rated"]):
        print(result)
    

start to load datasets
start cache
cache dataset complete
load user similarity matrix from cache
similarity matrix computed/loaded
start to predict user<1>'s rating for movie<2>
predict user<1> rating for movie<2>: 3.38
(1, 2, 3.38)
start to predict user<1>'s rating for movie<122882>
predict user<1> rating for movie<122882>: 3.74
(1, 122882, 3.74)
start to predict user<1>'s rating for movie<5>
predict user<1> rating for movie<5>: 3.09
(1, 5, 3.09)
start to predict user<1>'s rating for movie<122886>
predict user<1> rating for movie<122886>: 3.95
(1, 122886, 3.95)
start to predict user<1>'s rating for movie<7>
predict user<1> rating for movie<7>: 3.22
(1, 7, 3.22)
start to predict user<1>'s rating for movie<9>
predict user<1> rating for movie<9>: 2.59
(1, 9, 2.59)
start to predict user<1>'s rating for movie<10>
predict user<1> rating for movie<10>: 3.41
(1, 10, 3.41)
start to predict user<1>'s rating for movie<11>
predict user<1> rating for movie<11>: 3.68
(1, 11, 3.68)
start to predict 

predict user<1> rating for movie<164>: 2.57
(1, 164, 2.57)
start to predict user<1>'s rating for movie<165>
predict user<1> rating for movie<165>: 3.44
(1, 165, 3.44)
start to predict user<1>'s rating for movie<168>
predict user<1> rating for movie<168>: 3.20
(1, 168, 3.2)
start to predict user<1>'s rating for movie<8360>
predict user<1> rating for movie<8360>: 3.50
(1, 8360, 3.5)
start to predict user<1>'s rating for movie<170>
predict user<1> rating for movie<170>: 3.12
(1, 170, 3.12)
start to predict user<1>'s rating for movie<8361>
predict user<1> rating for movie<8361>: 3.00
(1, 8361, 3.0)
start to predict user<1>'s rating for movie<172>
predict user<1> rating for movie<172>: 2.29
(1, 172, 2.29)
start to predict user<1>'s rating for movie<173>
predict user<1> rating for movie<173>: 2.26
(1, 173, 2.26)
start to predict user<1>'s rating for movie<8366>
predict user<1> rating for movie<8366>: 3.74
(1, 8366, 3.74)
start to predict user<1>'s rating for movie<175>
predict user<1> rating

predict user<1> rating for movie<319>: 3.96
(1, 319, 3.96)
start to predict user<1>'s rating for movie<41285>
predict user<1> rating for movie<41285>: 3.66
(1, 41285, 3.66)
start to predict user<1>'s rating for movie<57669>
predict user<1> rating for movie<57669>: 4.22
(1, 57669, 4.22)
start to predict user<1>'s rating for movie<327>
predict user<1> rating for movie<327>: 2.45
(1, 327, 2.45)
start to predict user<1>'s rating for movie<329>
predict user<1> rating for movie<329>: 3.30
(1, 329, 3.3)
start to predict user<1>'s rating for movie<332>
predict user<1> rating for movie<332>: 2.89
(1, 332, 2.89)
start to predict user<1>'s rating for movie<8528>
predict user<1> rating for movie<8528>: 3.39
(1, 8528, 3.39)
start to predict user<1>'s rating for movie<337>
predict user<1> rating for movie<337>: 3.73
(1, 337, 3.73)
start to predict user<1>'s rating for movie<338>
predict user<1> rating for movie<338>: 2.36
(1, 338, 2.36)
start to predict user<1>'s rating for movie<339>
predict user<1

predict user<1> rating for movie<508>: 3.62
(1, 508, 3.62)
start to predict user<1>'s rating for movie<509>
predict user<1> rating for movie<509>: 3.55
(1, 509, 3.55)
start to predict user<1>'s rating for movie<514>
predict user<1> rating for movie<514>: 3.27
(1, 514, 3.27)
start to predict user<1>'s rating for movie<515>
predict user<1> rating for movie<515>: 3.80
(1, 515, 3.8)
start to predict user<1>'s rating for movie<516>
predict user<1> rating for movie<516>: 3.15
(1, 516, 3.15)
start to predict user<1>'s rating for movie<519>
predict user<1> rating for movie<519>: 2.01
(1, 519, 2.01)
start to predict user<1>'s rating for movie<520>
predict user<1> rating for movie<520>: 3.19
(1, 520, 3.19)
start to predict user<1>'s rating for movie<115210>
predict user<1> rating for movie<115210>: 3.75
(1, 115210, 3.75)
start to predict user<1>'s rating for movie<524>
predict user<1> rating for movie<524>: 3.46
(1, 524, 3.46)
start to predict user<1>'s rating for movie<529>
predict user<1> rati

predict user<1> rating for movie<765>: 3.08
(1, 765, 3.08)
start to predict user<1>'s rating for movie<8957>
predict user<1> rating for movie<8957>: 3.29
(1, 8957, 3.29)
start to predict user<1>'s rating for movie<8958>
predict user<1> rating for movie<8958>: 3.84
(1, 8958, 3.84)
start to predict user<1>'s rating for movie<8961>
predict user<1> rating for movie<8961>: 3.83
(1, 8961, 3.83)
start to predict user<1>'s rating for movie<8965>
predict user<1> rating for movie<8965>: 3.07
(1, 8965, 3.07)
start to predict user<1>'s rating for movie<8969>
predict user<1> rating for movie<8969>: 2.91
(1, 8969, 2.91)
start to predict user<1>'s rating for movie<778>
predict user<1> rating for movie<778>: 4.14
(1, 778, 4.14)
start to predict user<1>'s rating for movie<8970>
predict user<1> rating for movie<8970>: 3.97
(1, 8970, 3.97)
start to predict user<1>'s rating for movie<8972>
predict user<1> rating for movie<8972>: 3.03
(1, 8972, 3.03)
start to predict user<1>'s rating for movie<783>
predict

...

predict user<1> rating for movie<54190>: 4.04
(1, 54190, 4.04)
start to predict user<1>'s rating for movie<5049>
predict user<1> rating for movie<5049>: 3.40
(1, 5049, 3.4)
start to predict user<1>'s rating for movie<95167>
predict user<1> rating for movie<95167>: 3.59
(1, 95167, 3.59)
start to predict user<1>'s rating for movie<5064>
predict user<1> rating for movie<5064>: 3.84
(1, 5064, 3.84)
start to predict user<1>'s rating for movie<5065>
predict user<1> rating for movie<5065>: 3.00
(1, 5065, 3.0)
start to predict user<1>'s rating for movie<5066>
predict user<1> rating for movie<5066>: 3.11
(1, 5066, 3.11)
start to predict user<1>'s rating for movie<5072>
predict user<1> rating for movie<5072>: 3.90
(1, 5072, 3.9)
start to predict user<1>'s rating for movie<62434>
predict user<1> rating for movie<62434>: 3.80
(1, 62434, 3.8)
start to predict user<1>'s rating for movie<5103>
predict user<1> rating for movie<5103>: 3.27
(1, 5103, 3.27)
start to predict user<1>'s rating for movie<542

predict user<1> rating for movie<5630>: 3.56
(1, 5630, 3.56)
start to predict user<1>'s rating for movie<112138>
predict user<1> rating for movie<112138>: 3.74
(1, 112138, 3.74)
start to predict user<1>'s rating for movie<5650>
predict user<1> rating for movie<5650>: 3.78
(1, 5650, 3.78)
start to predict user<1>'s rating for movie<62999>
predict user<1> rating for movie<62999>: 3.74
(1, 62999, 3.74)
start to predict user<1>'s rating for movie<5669>
predict user<1> rating for movie<5669>: 3.81
(1, 5669, 3.81)
start to predict user<1>'s rating for movie<5673>
predict user<1> rating for movie<5673>: 3.77
(1, 5673, 3.77)
start to predict user<1>'s rating for movie<5679>
predict user<1> rating for movie<5679>: 3.08
(1, 5679, 3.08)
start to predict user<1>'s rating for movie<112175>
predict user<1> rating for movie<112175>: 4.35
(1, 112175, 4.35)
start to predict user<1>'s rating for movie<112183>
predict user<1> rating for movie<112183>: 3.56
(1, 112183, 3.56)
start to predict user<1>'s rat

predict user<1> rating for movie<6218>: 3.25
(1, 6218, 3.25)
start to predict user<1>'s rating for movie<88140>
predict user<1> rating for movie<88140>: 3.41
(1, 88140, 3.41)
start to predict user<1>'s rating for movie<30810>
predict user<1> rating for movie<30810>: 3.74
(1, 30810, 3.74)
start to predict user<1>'s rating for movie<30812>
predict user<1> rating for movie<30812>: 3.49
(1, 30812, 3.49)
start to predict user<1>'s rating for movie<30816>
predict user<1> rating for movie<30816>: 3.23
(1, 30816, 3.23)
start to predict user<1>'s rating for movie<47200>
predict user<1> rating for movie<47200>: 3.09
(1, 47200, 3.09)
start to predict user<1>'s rating for movie<6242>
predict user<1> rating for movie<6242>: 3.58
(1, 6242, 3.58)
start to predict user<1>'s rating for movie<88163>
predict user<1> rating for movie<88163>: 3.99
(1, 88163, 3.99)
start to predict user<1>'s rating for movie<30822>
predict user<1> rating for movie<30822>: 3.46
(1, 30822, 3.46)
start to predict user<1>'s rat

predict user<1> rating for movie<6863>: 3.35
(1, 6863, 3.35)
start to predict user<1>'s rating for movie<6867>
predict user<1> rating for movie<6867>: 3.99
(1, 6867, 3.99)
start to predict user<1>'s rating for movie<6870>
predict user<1> rating for movie<6870>: 3.73
(1, 6870, 3.73)
start to predict user<1>'s rating for movie<6873>
predict user<1> rating for movie<6873>: 3.10
(1, 6873, 3.1)
start to predict user<1>'s rating for movie<6874>
predict user<1> rating for movie<6874>: 3.99
(1, 6874, 3.99)
start to predict user<1>'s rating for movie<6879>
predict user<1> rating for movie<6879>: 3.57
(1, 6879, 3.57)
start to predict user<1>'s rating for movie<6888>
predict user<1> rating for movie<6888>: 2.10
(1, 6888, 2.1)
start to predict user<1>'s rating for movie<88810>
predict user<1> rating for movie<88810>: 3.69
(1, 88810, 3.69)
start to predict user<1>'s rating for movie<6934>
predict user<1> rating for movie<6934>: 3.28
(1, 6934, 3.28)
start to predict user<1>'s rating for movie<6936>


predict user<1> rating for movie<73017>: 3.97
(1, 73017, 3.97)
start to predict user<1>'s rating for movie<7482>
predict user<1> rating for movie<7482>: 4.00
(1, 7482, 4.0)
start to predict user<1>'s rating for movie<64839>
predict user<1> rating for movie<64839>: 3.84
(1, 64839, 3.84)
start to predict user<1>'s rating for movie<81229>
predict user<1> rating for movie<81229>: 3.10
(1, 81229, 3.1)
start to predict user<1>'s rating for movie<40278>
predict user<1> rating for movie<40278>: 3.74
(1, 40278, 3.74)
start to predict user<1>'s rating for movie<105844>
predict user<1> rating for movie<105844>: 3.86
(1, 105844, 3.86)
start to predict user<1>'s rating for movie<89470>
predict user<1> rating for movie<89470>: 3.80
(1, 89470, 3.8)
start to predict user<1>'s rating for movie<48516>
predict user<1> rating for movie<48516>: 4.44
(1, 48516, 4.44)
start to predict user<1>'s rating for movie<7569>
predict user<1> rating for movie<7569>: 3.56
(1, 7569, 3.56)
start to predict user<1>'s rati

In [73]:
def top_k_rs_result(k):
    ratings_matrix = load_data(data_path)
    user_similar = compute_pearson_similarity(ratings_matrix, based="user")
    results = predict_all(1, ratings_matrix, user_similar, filter_rule=["unhot", "rated"])
    return sorted(results, key=lambda x: x[2], reverse=True)[:k]

if __name__ == '__main__':
    from pprint import pprint
    result = top_k_rs_result(20)
    pprint(result)

start to load datasets
start cache
cache dataset complete
load user similarity matrix from cache
similarity matrix computed/loaded
start to predict user<1>'s rating for movie<2>
predict user<1> rating for movie<2>: 3.38
start to predict user<1>'s rating for movie<122882>
predict user<1> rating for movie<122882>: 3.74
start to predict user<1>'s rating for movie<5>
predict user<1> rating for movie<5>: 3.09
start to predict user<1>'s rating for movie<122886>
predict user<1> rating for movie<122886>: 3.95
start to predict user<1>'s rating for movie<7>
predict user<1> rating for movie<7>: 3.22
start to predict user<1>'s rating for movie<9>
predict user<1> rating for movie<9>: 2.59
start to predict user<1>'s rating for movie<10>
predict user<1> rating for movie<10>: 3.41
start to predict user<1>'s rating for movie<11>
predict user<1> rating for movie<11>: 3.68
start to predict user<1>'s rating for movie<12>
predict user<1> rating for movie<12>: 2.49
start to predict user<1>'s rating for movi

start to predict user<1>'s rating for movie<170>
predict user<1> rating for movie<170>: 3.12
start to predict user<1>'s rating for movie<8361>
predict user<1> rating for movie<8361>: 3.00
start to predict user<1>'s rating for movie<172>
predict user<1> rating for movie<172>: 2.29
start to predict user<1>'s rating for movie<173>
predict user<1> rating for movie<173>: 2.26
start to predict user<1>'s rating for movie<8366>
predict user<1> rating for movie<8366>: 3.74
start to predict user<1>'s rating for movie<175>
predict user<1> rating for movie<175>: 3.99
start to predict user<1>'s rating for movie<176>
predict user<1> rating for movie<176>: 4.31
start to predict user<1>'s rating for movie<8368>
predict user<1> rating for movie<8368>: 4.01
start to predict user<1>'s rating for movie<8370>
predict user<1> rating for movie<8370>: 4.07
start to predict user<1>'s rating for movie<8371>
predict user<1> rating for movie<8371>: 2.82
start to predict user<1>'s rating for movie<180>
predict use

predict user<1> rating for movie<8533>: 3.78
start to predict user<1>'s rating for movie<342>
predict user<1> rating for movie<342>: 3.43
start to predict user<1>'s rating for movie<164179>
predict user<1> rating for movie<164179>: 3.84
start to predict user<1>'s rating for movie<344>
predict user<1> rating for movie<344>: 2.99
start to predict user<1>'s rating for movie<345>
predict user<1> rating for movie<345>: 3.62
start to predict user<1>'s rating for movie<348>
predict user<1> rating for movie<348>: 3.64
start to predict user<1>'s rating for movie<350>
predict user<1> rating for movie<350>: 3.07
start to predict user<1>'s rating for movie<351>
predict user<1> rating for movie<351>: 3.22
start to predict user<1>'s rating for movie<353>
predict user<1> rating for movie<353>: 3.90
start to predict user<1>'s rating for movie<355>
predict user<1> rating for movie<355>: 1.99
start to predict user<1>'s rating for movie<357>
predict user<1> rating for movie<357>: 3.55
start to predict us

predict user<1> rating for movie<519>: 2.01
start to predict user<1>'s rating for movie<520>
predict user<1> rating for movie<520>: 3.19
start to predict user<1>'s rating for movie<115210>
predict user<1> rating for movie<115210>: 3.75
start to predict user<1>'s rating for movie<524>
predict user<1> rating for movie<524>: 3.46
start to predict user<1>'s rating for movie<529>
predict user<1> rating for movie<529>: 3.81
start to predict user<1>'s rating for movie<531>
predict user<1> rating for movie<531>: 3.42
start to predict user<1>'s rating for movie<532>
predict user<1> rating for movie<532>: 2.98
start to predict user<1>'s rating for movie<533>
predict user<1> rating for movie<533>: 2.68
start to predict user<1>'s rating for movie<534>
predict user<1> rating for movie<534>: 3.93
start to predict user<1>'s rating for movie<535>
predict user<1> rating for movie<535>: 3.03
start to predict user<1>'s rating for movie<537>
predict user<1> rating for movie<537>: 2.98
start to predict use

predict user<1> rating for movie<778>: 4.14
start to predict user<1>'s rating for movie<8970>
predict user<1> rating for movie<8970>: 3.97
start to predict user<1>'s rating for movie<8972>
predict user<1> rating for movie<8972>: 3.03
start to predict user<1>'s rating for movie<783>
predict user<1> rating for movie<783>: 3.35
start to predict user<1>'s rating for movie<784>
predict user<1> rating for movie<784>: 2.76
start to predict user<1>'s rating for movie<785>
predict user<1> rating for movie<785>: 3.51
start to predict user<1>'s rating for movie<786>
predict user<1> rating for movie<786>: 2.83
start to predict user<1>'s rating for movie<788>
predict user<1> rating for movie<788>: 2.50
start to predict user<1>'s rating for movie<8981>
predict user<1> rating for movie<8981>: 3.50
start to predict user<1>'s rating for movie<8983>
predict user<1> rating for movie<8983>: 3.51
start to predict user<1>'s rating for movie<8984>
predict user<1> rating for movie<8984>: 3.19
start to predict

...

predict user<1> rating for movie<4963>: 3.81
start to predict user<1>'s rating for movie<37733>
predict user<1> rating for movie<37733>: 3.45
start to predict user<1>'s rating for movie<4967>
predict user<1> rating for movie<4967>: 4.32
start to predict user<1>'s rating for movie<4973>
predict user<1> rating for movie<4973>: 4.23
start to predict user<1>'s rating for movie<4974>
predict user<1> rating for movie<4974>: 2.40
start to predict user<1>'s rating for movie<4975>
predict user<1> rating for movie<4975>: 3.41
start to predict user<1>'s rating for movie<37741>
predict user<1> rating for movie<37741>: 3.90
start to predict user<1>'s rating for movie<4979>
predict user<1> rating for movie<4979>: 3.82
start to predict user<1>'s rating for movie<4980>
predict user<1> rating for movie<4980>: 2.93
start to predict user<1>'s rating for movie<45950>
predict user<1> rating for movie<45950>: 3.58
start to predict user<1>'s rating for movie<86911>
predict user<1> rating for movie<86911>: 3.

predict user<1> rating for movie<5620>: 3.03
start to predict user<1>'s rating for movie<5630>
predict user<1> rating for movie<5630>: 3.56
start to predict user<1>'s rating for movie<112138>
predict user<1> rating for movie<112138>: 3.74
start to predict user<1>'s rating for movie<5650>
predict user<1> rating for movie<5650>: 3.78
start to predict user<1>'s rating for movie<62999>
predict user<1> rating for movie<62999>: 3.74
start to predict user<1>'s rating for movie<5669>
predict user<1> rating for movie<5669>: 3.81
start to predict user<1>'s rating for movie<5673>
predict user<1> rating for movie<5673>: 3.77
start to predict user<1>'s rating for movie<5679>
predict user<1> rating for movie<5679>: 3.08
start to predict user<1>'s rating for movie<112175>
predict user<1> rating for movie<112175>: 4.35
start to predict user<1>'s rating for movie<112183>
predict user<1> rating for movie<112183>: 3.56
start to predict user<1>'s rating for movie<5690>
predict user<1> rating for movie<569

predict user<1> rating for movie<30793>: 3.05
start to predict user<1>'s rating for movie<6218>
predict user<1> rating for movie<6218>: 3.25
start to predict user<1>'s rating for movie<88140>
predict user<1> rating for movie<88140>: 3.41
start to predict user<1>'s rating for movie<30810>
predict user<1> rating for movie<30810>: 3.74
start to predict user<1>'s rating for movie<30812>
predict user<1> rating for movie<30812>: 3.49
start to predict user<1>'s rating for movie<30816>
predict user<1> rating for movie<30816>: 3.23
start to predict user<1>'s rating for movie<47200>
predict user<1> rating for movie<47200>: 3.09
start to predict user<1>'s rating for movie<6242>
predict user<1> rating for movie<6242>: 3.58
start to predict user<1>'s rating for movie<88163>
predict user<1> rating for movie<88163>: 3.99
start to predict user<1>'s rating for movie<30822>
predict user<1> rating for movie<30822>: 3.46
start to predict user<1>'s rating for movie<30825>
predict user<1> rating for movie<3

predict user<1> rating for movie<6873>: 3.10
start to predict user<1>'s rating for movie<6874>
predict user<1> rating for movie<6874>: 3.99
start to predict user<1>'s rating for movie<6879>
predict user<1> rating for movie<6879>: 3.57
start to predict user<1>'s rating for movie<6888>
predict user<1> rating for movie<6888>: 2.10
start to predict user<1>'s rating for movie<88810>
predict user<1> rating for movie<88810>: 3.69
start to predict user<1>'s rating for movie<6934>
predict user<1> rating for movie<6934>: 3.28
start to predict user<1>'s rating for movie<6936>
predict user<1> rating for movie<6936>: 3.68
start to predict user<1>'s rating for movie<6942>
predict user<1> rating for movie<6942>: 3.81
start to predict user<1>'s rating for movie<6944>
predict user<1> rating for movie<6944>: 3.50
start to predict user<1>'s rating for movie<6947>
predict user<1> rating for movie<6947>: 3.58
start to predict user<1>'s rating for movie<6953>
predict user<1> rating for movie<6953>: 3.41
sta

predict user<1> rating for movie<105844>: 3.86
start to predict user<1>'s rating for movie<89470>
predict user<1> rating for movie<89470>: 3.80
start to predict user<1>'s rating for movie<48516>
predict user<1> rating for movie<48516>: 4.44
start to predict user<1>'s rating for movie<7569>
predict user<1> rating for movie<7569>: 3.56
start to predict user<1>'s rating for movie<89492>
predict user<1> rating for movie<89492>: 3.89
start to predict user<1>'s rating for movie<56757>
predict user<1> rating for movie<56757>: 3.75
start to predict user<1>'s rating for movie<64957>
predict user<1> rating for movie<64957>: 3.67
start to predict user<1>'s rating for movie<56775>
predict user<1> rating for movie<56775>: 2.88
start to predict user<1>'s rating for movie<64969>
predict user<1> rating for movie<64969>: 3.82
start to predict user<1>'s rating for movie<56782>
predict user<1> rating for movie<56782>: 4.14
start to predict user<1>'s rating for movie<64983>
predict user<1> rating for movi